In [ ]:
pip install openvino-dev

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 85.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 87.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 108.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for jstyleson: filename=jstyleson-0.0.2-py3-none-any.whl size=2398 sha256=7e6e5f275f1f5d90b03f296c79932e654a144965cb46746a39e961f2bdb11da1
  Stored in directory: /root/.cache/pip/wheels/d4/2a/06/11202ea86be0f51f34e9411d691e25b991d188d93ab4d3e551
Successfully built jstyleson
  Attempting uninstall: networkx
    Found existing installation: networkx 3.1
    Uninstalling networkx-3.1:
      Successfully uninstalled networkx-3.1
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency 

In [ ]:
import abc

import numpy as np
from PIL import Image


class FeatureExtractor(abc.ABC):
    @abc.abstractmethod
    def extract_features(self, input_image: Image) -> np.ndarray:
        pass

In [ ]:
import numpy as np
import tensorflow as tf
from PIL import Image
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image


class ResNetExtractor(FeatureExtractor):
    def __init__(self):
        base_model = tf.keras.applications.ResNet50(
            include_top=True, weights="imagenet"
        )
        self.model = Model(
            inputs=base_model.input, outputs=base_model.get_layer("avg_pool").output
        )

    def extract_features(self, input_image: Image) -> np.ndarray:
        img = input_image.resize(
            (224, 224)
        )  # Resnet must take a 224x224 img as an input
        img = img.convert("RGB")  # Make sure img is color
        x = image.img_to_array(
            img
        )  # To np.array. Height x Width x Channel. dtype=float32
        x = np.expand_dims(
            x, axis=0
        )  # (H, W, C)->(1, H, W, C), where the first elem is the number of img
        x = preprocess_input(x)  # Subtracting avg values for each pixel
        feature = self.model.predict(x)[0]
        return feature / np.linalg.norm(feature)

In [ ]:
base_model = tf.keras.applications.ResNet50(
            include_top=True, weights="imagenet"
        )

102967424/102967424 [==============================] - 0s 0us/step


In [ ]:
model_path = 'resnet50'
model_path_name ='resnet50featuresss'

In [ ]:
base_model.save(model_path)

In [ ]:
base_model.get_layer("avg_pool")

In [ ]:
import time
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from IPython.display import Markdown
from openvino.runtime import Core

In [ ]:
# Construct the command for Model Optimizer.
mo_command = f"""mo
                 --saved_model_dir "{model_path}"
                 --input_shape "[1,224,224,3]"
                 --model_name "{model_path_name}"
                 --compress_to_fp16
                 --output 'StatefulPartitionedCall/resnet50/avg_pool/Mean'
                 """
mo_command = " ".join(mo_command.split())
print("Model Optimizer command to convert TensorFlow to OpenVINO:")
display(Markdown(f"`{mo_command}`"))

Model Optimizer command to convert TensorFlow to OpenVINO:


`mo --saved_model_dir "resnet50" --input_shape "[1,224,224,3]" --model_name "resnet50featuresss" --compress_to_fp16 --output 'StatefulPartitionedCall/resnet50/avg_pool/Mean'`

In [ ]:
ir_path = Path("resnet50features_openvino.xml")

In [ ]:
# Run Model Optimizer if the IR model file does not exist
if not ir_path.exists():
    print("Exporting TensorFlow model to IR... This may take a few minutes.")
    ! $mo_command
else:
    print(f"IR model {ir_path} already exists.")

Exporting TensorFlow model to IR... This may take a few minutes.
[ INFO ] The model was converted to IR v11, the latest model format that corresponds to the source DL framework input/output format. While IR v11 is backwards compatible with OpenVINO Inference Engine API v1.0, please use API v2.0 (as of 2022.1) to take advantage of the latest improvements in IR v11.
Find more information about API v2.0 and IR v11 at https://docs.openvino.ai/latest/openvino_2_0_transition_guide.html
[ SUCCESS ] Generated IR version 11 model.
[ SUCCESS ] XML file: /content/resnet50featuresss.xml
[ SUCCESS ] BIN file: /content/resnet50featuresss.bin


/bin/bash: mo: command not found


In [ ]:
ie = Core()
model = ie.read_model('resnet50featuresss.xml')
compiled_model = ie.compile_model(model=model, device_name="CPU")

In [ ]:
input_key = compiled_model.input(0)
output_key = compiled_model.output(0)
network_input_shape = input_key.shape 

In [ ]:
# The MobileNet network expects images in RGB format.
image = cv2.cvtColor(cv2.imread(filename="/content/test.jpeg"), code=cv2.COLOR_BGR2RGB)

# Resize the image to the network input shape.
resized_image = cv2.resize(src=image, dsize=(224, 224))

# Transpose the image to the network input shape.
input_image = np.expand_dims(resized_image, 0)

plt.imshow(image);

In [ ]:
result = compiled_model(input_image)[output_key]

In [ ]:
result

array([[0.2416449 , 0.86873645, 0.2758638 , ..., 0.        , 0.        ,
        2.319537  ]], dtype=float32)

In [ ]:
np.shape(result)

(1, 2048)